In [1]:
from process.clinic_data import writeClinic
from process.img_data import imgRedir
from process.pat_data import combinePatData
from src.feature.base import genROI
from src.utils.data import getPandas
import os
os.chdir('..')

In [2]:
os.getcwd()

'/home/biobot/disk/ldopa_pred'

In [3]:
data = getPandas('pat_data')